<a href="https://colab.research.google.com/github/snbigft/update_postgres_colum_from_python/blob/main/Update_Postgres_colum_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -U torch torchvision sentence-transformers psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 753.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.1/150.1 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import psycopg2
from sentence_transformers import SentenceTransformer

# Configuración de la base de datos
DB_HOST = "ep-snowy-dawn-a8omlpke-pooler.eastus2.azure.neon.tech"
DB_NAME = "neondb"
DB_USER = "neondb_owner"
DB_PASSWORD = "npg_BcpnId62FYhS"

# Función para conectar a la base de datos PostgreSQL
def get_db_connection():
    return psycopg2.connect(
        host=DB_HOST,
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        sslmode="require"
    )

# Cargar el modelo de embeddings (Sentence Transformers)
model = SentenceTransformer('all-MiniLM-L6-v2')

# Paso 1: Añadir la columna vector_data si no existe
def add_vector_column():
    try:
        conn = get_db_connection()
        cursor = conn.cursor()
        cursor.execute("""
            ALTER TABLE Customers
            ADD COLUMN IF NOT EXISTS vector_data real[];
        """)
        conn.commit()
        cursor.close()
        conn.close()
        print("✅ Columna 'vector_data' añadida correctamente.")
    except Exception as e:
        print(f"❌ Error añadiendo la columna: {e}")

# Paso 2: Vectorizar y actualizar los datos
# Paso 2: Vectorizar y actualizar los datos utilizando ctid
def update_vector_data():
    try:
        conn = get_db_connection()
        cursor = conn.cursor()

        # Obtener los datos actuales con ctid
        cursor.execute("SELECT ctid, customer_data FROM Customers WHERE vector_data IS NULL")
        rows = cursor.fetchall()

        for row in rows:
            row_id = row[0]  # ctid se usa como identificador único
            customer_data = row[1]

            # Convertir el JSON a una cadena para la vectorización
            if isinstance(customer_data, dict):
                text_data = " ".join([f"{k}: {v}" for k, v in customer_data.items()])
            else:
                text_data = str(customer_data)

            # Generar el vector
            vector = model.encode(text_data).tolist()

            # Actualizar la base de datos con el vector usando ctid
            cursor.execute(
                "UPDATE Customers SET vector_data = %s WHERE ctid = %s",
                (vector, row_id)
            )

        conn.commit()
        cursor.close()
        conn.close()
        print("✅ Datos vectorizados y actualizados correctamente.")
    except Exception as e:
        print(f"❌ Error al vectorizar datos: {e}")

# Ejecutar las funciones
add_vector_column()
update_vector_data()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Columna 'vector_data' añadida correctamente.
✅ Datos vectorizados y actualizados correctamente.
